# Best poet 

<p style="font-size: 20px; font-weight: bold; font-style: italic;">...via LLM-graph</p>

Anton Antonov   
August 2025

---

## Introduction

This notebook illustrates how to specify a Large Language Model (LLM) graph using different LLM personas representing four poets and a poem judge.
The Raku package ["LLM::Graph"](https://raku.land/zef:antononcube/LLM::Graph) is used, [AAp1].

(The poet-judge examples are inspired from the Wolfram Language function page of [`LLMGraph`](https://reference.wolfram.com/language/ref/LLMGraph.html), [WRIf1].)

----

## Setup

In [13]:
use LLM::Functions;
use LLM::Graph;

In [14]:
sink my $conf5 = llm-configuration('ChatGPT', model => 'gpt-5');
sink my $conf5-mini = llm-configuration('ChatGPT', model => 'gpt-5-mini');
sink my $conf5-nano = llm-configuration('ChatGPT', model => 'gpt-5-nano');
sink my $conf41 = llm-configuration('ChatGPT', model => 'gpt-4.1', temperature => 0.55, max-tokens => 8192);
sink my $conf41-mini = llm-configuration('ChatGPT', model => 'gpt-4.1-mini', temperature => 0.55, max-tokens => 8192);
sink my $conf-gemini-pro = llm-configuration('Gemini', model => 'gemini-2.5-pro', temperature => 0.55, max-tokens => 8192);
sink my $conf-gemini-flash = llm-configuration('Gemini', model => 'gemini-2.5-flash', temperature => 0.55, max-tokens => 8192);

In [15]:
sink my $conf-ollama = LLM::Functions::Configuration.new(
        name => 'ollama',
        api-key => Whatever,
        api-user-id => 'user:' ~ ((10 ** 11 + 1) .. 10 ** 12).pick,
        module => 'WWW::Ollama',
        base-url => ollama-base-url,
        model => 'gemma3:12b',
        function => &ollama-chat-completion,
        embedding-model => 'nomic-embed-text',
        embedding-function => &ollama-embedding,
        temperature => 0.4,
        max-tokens => 8192,
        total-probability-cutoff => 0,
        prompts => Empty,
        prompt-delimiter => ' ',
        examples => Empty,
        stop-tokens => Empty,
        argument-renames => %(),
        format => 'values');

In [ ]:
sink my $conf = $conf41-mini;

-----

## LLM graph

Nodes spec:

In [17]:
sink my %rules =
    poet1 => "Write a short poem about summer.",
    poet2 => "Write a haiku about winter.",
    poet3 => sub ($topic, $style) {"Write a poem about $topic in the style $style."},
    judge => sub ($poet1, $poet2, $poet3) {
        ["Choose the best composition of your among these:\n\n",
        "1) Poem1: $poet1",
        "2) Poem2: $poet2",
        "3) Poem3: $poet3", 
        "and copy it:"].join("\n\n") }
;

More complicated spec:

In [18]:
sink my %rules =
        poet1 => "Write a short poem about summer.",
        poet2 => "Write a haiku about winter.",
        poet3 => sub ($topic, $style) {
            "Write a poem about $topic in the $style style."
        },
        poet4 => {
                eval-function => {llm-synthesize('You are a famous Russian poet. Write a short poem about playing bears.', llm-evaluator => $conf)},
                test-function => -> $with-russian { $with-russian ~~ Bool:D && $with-russian || $with-russian.Str.lc ∈ <true yes> }
        },
        judge => sub ($poet1, $poet2, $poet3, $poet4) {
            [
                "Choose the composition you think is best among these:\n\n",
                "1) Poem1: $poet1",
                "2) Poem2: $poet2",
                "3) Poem3: {$poet4.defined && $poet4 ?? $poet4 !! $poet3}",
                "and copy it:"
            ].join("\n\n")
        };

Make the graph:

In [19]:
my $gBestPoem = LLM::Graph.new(%rules, llm-evaluator => $conf):async

LLM::Graph(size => 5, nodes => judge, poet1, poet2, poet3, poet4)

----

## Full computation

Run the graph with specified parameters:

In [20]:
$gBestPoem.eval({topic => 'Hockey', style => 'limerick', with-russian => 'yes'})

LLM::Graph(size => 5, nodes => judge, poet1, poet2, poet3, poet4)

Show the corresponding graph-plot:

In [21]:
#% html
$gBestPoem.dot(engine => 'dot', :9graph-size, node-width => 1.2, theme => 'default'):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.1 (20251213.1925)
 -->
<!-- Pages: 1 -->
 
 
 
 
 poet3_Cluster 
 
 
 
 judge_Cluster 
 
 
<!-- $_ -->
 
 $_ 
 
 $_ 
 
<!-- poet1 -->
 
 poet1 
<polygon fill="SteelBlue" stroke="Black" points="50.25,-73.63 53.06,-73.69 55.85,-73.78 58.6,-73.89 61.3,-74.04 63.93,-74.22 66.49,-74.42 68.97,-74.65 71.35,-74.91 73.64,-75.19 75.81,-75.5 77.87,-75.83 79.81,-76.18 81.61,-76.56 83.28,-76.96 84.81,-77.38 86.2,-77.82 87.44,-78.28 88.53,-78.76 89.47,-79.24 90.26,-79.75 90.9,-80.27 91.39,-80.79 91.73,-81.33 91.93,-81.88 91.98,-82.43 91.89,-82.99 91.67,-83.55 91.32,-84.12 90.84,-84.68 90.24,-85.25 89.52,-85.81 88.69,-86.37 87.76,-86.92 86.74,-87.47 85.62,-88.01 84.42,-88.53 83.14,-89.05 81.78,-89.56 80.36,-90.04 78.88,-90.52 77.34,-90.98 75.76,-91.42 74.13,-91.84 72.46,-92.24 70.75,-92.62 69.02,-92.97 67.26,-93.3 65.47,-93.61 63.67,-93.89 61.84,-94.15 60.01,-94.38 58.16,-94.58 56.3,-94.76 54.44,-94.91 52.57,-95.02 50.69,-95.11 48.81,-95.17 46.93,-95.2 45.05,-95.2 43.17,-95.17 41.29,-95.11 39.41,-95.02 37.54,-94.91 35.68,-94.76 33.82,-94.58 31.97,-94.38 30.13,-94.15 28.31,-93.89 26.51,-93.61 24.72,-93.3 22.96,-92.97 21.23,-92.62 19.52,-92.24 17.85,-91.84 16.22,-91.42 14.64,-90.98 13.1,-90.52 11.62,-90.04 10.2,-89.56 8.84,-89.05 7.56,-88.53 6.36,-88.01 5.24,-87.47 4.21,-86.92 3.29,-86.37 2.46,-85.81 1.74,-85.25 1.14,-84.68 0.66,-84.12 0.31,-83.55 0.09,-82.99 0,-82.43 0.05,-81.88 0.25,-81.33 0.59,-80.79 1.08,-80.27 1.72,-79.75 2.51,-79.24 3.45,-78.76 4.54,-78.28 5.78,-77.82 7.17,-77.38 8.7,-76.96 10.37,-76.56 12.17,-76.18 14.11,-75.83 16.16,-75.5 18.34,-75.19 20.62,-74.91 23.01,-74.65 25.49,-74.42 28.05,-74.22 30.68,-74.04 33.38,-73.89 36.13,-73.78 38.92,-73.69 41.73,-73.63 44.57,-73.6 47.41,-73.6 50.25,-73.63"/>
 poet1 
 
<!-- $_->poet1 -->
 
 $_->poet1 
 
 
 
<!-- poet2 -->
 
 poet2 
<polygon fill="SteelBlue" stroke="Black" points="160.25,-73.63 163.06,-73.69 165.85,-73.78 168.6,-73.89 171.3,-74.04 173.93,-74.22 176.49,-74.42 178.97,-74.65 181.35,-74.91 183.64,-75.19 185.81,-75.5 187.87,-75.83 189.81,-76.18 191.61,-76.56 193.28,-76.96 194.81,-77.38 196.2,-77.82 197.44,-78.28 198.53,-78.76 199.47,-79.24 200.26,-79.75 200.9,-80.27 201.39,-80.79 201.73,-81.33 201.93,-81.88 201.98,-82.43 201.89,-82.99 201.67,-83.55 201.32,-84.12 200.84,-84.68 200.24,-85.25 199.52,-85.81 198.69,-86.37 197.76,-86.92 196.74,-87.47 195.62,-88.01 194.42,-88.53 193.14,-89.05 191.78,-89.56 190.36,-90.04 188.88,-90.52 187.34,-90.98 185.76,-91.42 184.13,-91.84 182.46,-92.24 180.75,-92.62 179.02,-92.97 177.26,-93.3 175.47,-93.61 173.67,-93.89 171.84,-94.15 170.01,-94.38 168.16,-94.58 166.3,-94.76 164.44,-94.91 162.57,-95.02 160.69,-95.11 158.81,-95.17 156.93,-95.2 155.05,-95.2 153.17,-95.17 151.29,-95.11 149.41,-95.02 147.54,-94.91 145.68,-94.76 143.82,-94.58 141.97,-94.38 140.13,-94.15 138.31,-93.89 136.51,-93.61 134.72,-93.3 132.96,-92.97 131.23,-92.62 129.52,-92.24 127.85,-91.84 126.22,-91.42 124.64,-90.98 123.1,-90.52 121.62,-90.04 120.2,-89.56 118.84,-89.05 117.56,-88.53 116.36,-88.01 115.24,-87.47 114.21,-86.92 113.29,-86.37 112.46,-85.81 111.74,-85.25 111.14,-84.68 110.66,-84.12 110.31,-83.55 110.09,-82.99 110,-82.43 110.05,-81.88 110.25,-81.33 110.59,-80.79 111.08,-80.27 111.72,-79.75 112.51,-79.24 113.45,-78.76 114.54,-78.28 115.78,-77.82 117.17,-77.38 118.7,-76.96 120.37,-76.56 122.17,-76.18 124.11,-75.83 126.16,-75.5 128.34,-75.19 130.62,-74.91 133.01,-74.65 135.49,-74.42 138.05,-74.22 140.68,-74.04 143.38,-73.89 146.13,-73.78 148.92,-73.69 151.73,-73.63 154.57,-73.6 157.41,-73.6 160.25,-73.63"/>
 poet2 
 
<!-- $_->poet2 -->
 
 $_->poet2 
 
 
 
<!-- poet4 -->
 
 poet4 
 
 poet4 
 
<!-- $_->poet4 -->
 
 $_->poet4 
 
 
 
<!-- judge -->
 
 judge 
 
 judge 
 
<!-- poet1->judge -->
 
 poet1->judge 
 
 
 
<!-- poet2->judge -->
 
 poet2->judge 
 
 
 
<!-- poet3 -->
 
 poet3 
 

In [22]:
#% html
$gBestPoem.dot(engine => 'dot', :9graph-size, node-width => 1.2, theme => 'ortho'):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.1 (20251213.1925)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- $_ -->
 
 $_ 
 
 $_ 
 
<!-- poet1 -->
 
 poet1 
 
 
 
 poet1 
 
<!-- $_->poet1 -->
 
 $_->poet1 
 
 
 
<!-- poet2 -->
 
 poet2 
 
 
 
 poet2 
 
<!-- $_->poet2 -->
 
 $_->poet2 
 
 
 
<!-- poet4 -->
 
 poet4 
 
 poet4 
 
<!-- $_->poet4 -->
 
 $_->poet4 
 
 
 
<!-- judge -->
 
 judge 
 
 
 judge 
 
<!-- poet1->judge -->
 
 poet1->judge 
 
 
 
<!-- poet2->judge -->
 
 poet2->judge 
 
 
 
<!-- poet3 -->
 
 poet3 
 
 
 poet3 
 
<!-- poet3->judge -->
 
 poet3->judge 
 
 
 
<!-- poet4->judge -->
 
 poet4->judge 
 
 
 
<!-- style -->
 
 style 
 
 style 
 
<!-- style->poet3 -->
 
 style->poet3 
 
 
 
<!-- topic -->
 
 topic 
 
 topic 
 
<!-- topic->poet3 -->
 
 topic->poet3 
 
 
 
<!-- with-russian -->
 
 with-russian 
 
 with-russian 
 
<!-- with-russian->poet4 -->
 
 with-russian->poet4

----

## Examine results

Show the "judge" node:

In [23]:
.say for |$gBestPoem.nodes<judge>

wrapper => Routine::WrapHandle.new
input => [poet2 poet3 poet4 poet1]
result => I think Poem1 is the best among these. It has a vivid, warm imagery and a pleasant rhythm that captures the joyful essence of summer beautifully.

Here is Poem1 copied:

Golden sun and skies so blue,  
Whispers warm in morning’s dew.  
Laughter dances on the breeze,  
Summer’s song among the trees.  

Fields of green and oceans wide,  
Endless days where dreams abide.  
Hearts alight with carefree cheer,  
Summer’s magic bright and clear.
spec-type => (Routine)
eval-function => sub { }
test-function-input => []


Tabulate node's results -- extract results from `Promise` objects if needed:

In [24]:
#% html
$gBestPoem.nodes
==> { .nodemap(*<result>) }()
==> { .nodemap({ $_ ~~ Promise:D ?? $_.result !! $_ }) }()
==> to-dataset()
==> { .sort(*<Key>) }()
==> to-html(align => 'left', field-names => <Key Value>)
==> { .subst("\n", "<br>", :g) }()

Key,Value
judge,"I think Poem1 is the best among these. It has a vivid, warm imagery and a pleasant rhythm that captures the joyful essence of summer beautifully.Here is Poem1 copied:Golden sun and skies so blue, Whispers warm in morning’s dew. Laughter dances on the breeze, Summer’s song among the trees. Fields of green and oceans wide, Endless days where dreams abide. Hearts alight with carefree cheer, Summer’s magic bright and clear."
poet1,"Golden sun and skies so blue, Whispers warm in morning’s dew. Laughter dances on the breeze, Summer’s song among the trees. Fields of green and oceans wide, Endless days where dreams abide. Hearts alight with carefree cheer, Summer’s magic bright and clear."
poet2,"Silent snowflakes fall, Blanketing the earth in white, Winter's breath is cold."
poet3,"There once was a game on the ice, Where skaters were swift and precise. With sticks they would chase, A puck in a race, Scoring goals was their prized device!"
poet4,"В чаще лесной, где шепчет трава, Играют медведи — весёлая семья. Пушистые лапы в танце кружат, Смех их в ночи, как звёзды, блуждают.Они не грозны, а просто играют, В мире лесном сказку создают. Медвежьи игры — радость и свет, В сердце природы живёт этот ответ."


-----

## References

[AAp1] Anton Antonov, 
[LLM::Graph, Raku package](https://github.com/antononcube/Raku-LLM-Graph),
(2025),
[GitHub/antononcube](https://github.com/antononcube).

[WRIf1] Wolfram Research (2025), 
[LLMGraph](https://reference.wolfram.com/language/ref/LLMGraph.html), 
[Wolfram Language function](https://reference.wolfram.com/language).